In [ ]:
# run for first time setup
! pip install ipywidgets
! jupyter nbextension enable --py widgetsnbextension

In [13]:
import sys
sys.path.append("..\\")
import login
login.connect()
import datajoint as dj

import ipywidgets as widgets
from ipywidgets import interact, interact_manual

In [29]:
import inspect
available_schema = dj.list_schemas()
schema_dropdown = widgets.Dropdown(options = available_schema, value = "common_mice")
dj_schema = dj.Schema("common_mouse")
mod = dj.VirtualModule("temp.py", "common_mice", create_schema = False, create_tables=False)

def get_table_names(virtualmodule):
    names = []
    for i in inspect.getmembers(virtualmodule):
        a = getattr(virtualmodule, i[0])
        if type(a) == dj.user_tables.TableMeta:
            names.append(i[0])
    return names

tables = get_table_names(mod)
table_dropdown = widgets.Dropdown(options = tables)

def schema_update(*args):
    mod = dj.VirtualModule("temp.py", schema_dropdown.value, create_schema = False, create_tables=False)
    tables = get_table_names(mod)
    print(tables)
    table_dropdown.options = tables
schema_dropdown.observe(schema_update, "value")

def get_table_attributes(table_class):
    print()
        
def show_table(schema, table):
    mod = dj.VirtualModule("temp.py", schema_dropdown.value, create_schema = False, create_tables=False)
    table_class = getattr(mod, table)
    return table_class()
    
interact(show_table, schema = schema_dropdown, table = table_dropdown)

interactive(children=(Dropdown(description='schema', index=3, options=('common_behav', 'common_exp', 'common_i…

<function __main__.show_table(schema, table)>

['MappingSession', 'RawParameterFile', 'RawSynchronisationFile', 'Stimulation']
['ActivityStatistics', 'BrainRegion', 'CaIndicator', 'CaimanParameter', 'Deconvolution', 'DeconvolutionModel', 'FieldOfViewSize', 'Laser', 'Layer', 'MemoryMappedFile', 'Microscope', 'MotionCorrection', 'MotionParameter', 'QualityControl', 'RawImagingFile', 'Scan', 'ScanInfo', 'Segmentation']
